In [1]:
from keys import keys

import json
import pandas as pd
import argparse
import io
import os
import requests
from datetime import date
import time

# XE api
from xecd_rates_client import XecdClient

# Google News api
from newsapi import NewsApiClient

# Google Cloud client library api
from google.cloud import language
from google.cloud.language import enums
from google.cloud.language import types


In [32]:
class Create_DF:
    def __init__(self, cur_original, cur_new, news_api_key, json_google_cloud, xecd_client, rapid_api_host, rapid_api_key):
        self.today = date.today().strftime("%Y/%m/%d").replace("/","-")
        self.original = cur_original
        self.new = cur_new
        self.newsapi = NewsApiClient(api_key=news_api_key)
        self.client = language.LanguageServiceClient.from_service_account_json(json_google_cloud)
        self.xecd = xecd_client
        self.rapid_api_host = rapid_api_host
        self.rapid_api_key = rapid_api_key
        self.approved_classifications = [
            'Finance',
            'Business',
            'Politics',
            'Jobs & Education',
            'Business News',
            'Business & Industrial',
        ]
        
        
    # classify text into categories   
    def __classify(self, text):
        client = self.client
        document = language.types.Document(content=text, type=language.enums.Document.Type.PLAIN_TEXT)
        response = client.classify_text(document)
        categories = response.categories

        result = []

        for category in categories:
            categories = category.name.split("/")
            categories.pop(0)
            for i in range(len(categories)):
                result.append(categories[i])
        return result
    
    def __getRate(self, date, fromCur, toCur):
        xecd = self.xecd
        rateResult = xecd.historic_rate(date, "12:00", fromCur, toCur, 1)
        rateObj={}
        rateObj['date'] = rateResult['timestamp'][:10]
        rateObj['rate'] = rateResult['to'][0]['mid']
        return rateObj['rate']
    
    
    # retrieve articles from google news api
    def __retrieve_articles(self):
        formatted_articles = []
        pageNum = 1
        
        print("===============Start Scraping===============")
        
        while True:
            url = "https://contextualwebsearch-websearch-v1.p.rapidapi.com/api/Search/NewsSearchAPI"
            querystring = {"fromPublishedDate":"2019-06-14 00:00:00","toPublishedDate":"2019-09-13 23:59:59","autoCorrect":"false","pageNumber":str(pageNum),"pageSize":"50","q":"USD United States","safeSearch":"false"}
            headers = {
                'x-rapidapi-host': self.rapid_api_host,
                'x-rapidapi-key': self.rapid_api_key
            }

            all_articles = requests.request("GET", url, headers=headers, params=querystring).json()
            iterationsLeft = (all_articles['totalCount'] - pageNum*50) // 50
            articles = all_articles['value']

            # creating new article object
            for article in articles:
                info = str(article['title']) + " " + str(article['description']) + " " + str(article['body'])
                classifications = self.__classify(info)
                if len(set(classifications).intersection(set(self.approved_classifications))) != 0:
                    individual_article = {}
                    individual_article['date'] = article['datePublished'][:10]

                    info = info.replace("'","") 
                    individual_article['text'] = info

                    formatted_articles.append(individual_article)
                    print("     Scraped")
    #         print(json.dumps(self.formatted_articles, indent=4, sort_keys=True))
            if iterationsLeft <= 0:
                print("Finished Scraping")
                break
            else:
                time.sleep(8)
                print("Scraped Page: ", pageNum)
                pageNum += 1
            
        self.formatted_articles = formatted_articles
    
    
    # Sentiment analysis using google cloud
    def __sentiment_analysis(self):
        client = self.client
        tone_analyzed_articles = {'date':[],'score':[]}
        formatted_articles = self.formatted_articles.copy()

        # The text to analyze
        for i in range(len(formatted_articles)):
            analyzed_article = {}
            article = formatted_articles[i]
            text = article['text']

            document = types.Document(content = text, type = enums.Document.Type.PLAIN_TEXT)

            # Detects the sentiment of the text
            sentiment = client.analyze_sentiment(document=document).document_sentiment
            
            # Appends to dataframe
            tone_analyzed_articles['date'].append(article['date'])
            tone_analyzed_articles['score'].append(sentiment.score)
        
#         print(json.dumps(tone_analyzed_articles, indent=4, sort_keys=True))        
        
        return pd.DataFrame(tone_analyzed_articles)
        
    # Dump the dataframe
    def __dump_df(self, df):
        df.to_csv(r'data/articles/' + self.original + '.csv')
        
        
    # Create the dataframe and save it
    def df_create(self):
        self.__retrieve_articles()
        df = self.__sentiment_analysis()
        self.__dump_df(df)
        return df

## Input Parameter

In [33]:
news_api_key = keys['news']
json_google_cloud = keys['json_google_cloud']
xecd_client = keys['xe']
rapid_api_host = keys['rapid_api_host']
rapid_api_key = keys['rapid_api_key']

In [34]:
print("Enter the original currency: ")
# cur_original = input()
print("Enter the new currency: ")
# cur_return = input()

Enter the original currency: 
Enter the new currency: 


In [35]:
cur_original = 'CAD'
cur_return = 'USD'

## Create Dataframe

In [38]:
created_df = Create_DF(cur_original, cur_return, news_api_key, json_google_cloud, xecd_client,rapid_api_host,rapid_api_key)

In [ ]:
df = created_df.df_create()

In [37]:
df.head(100)

,date,score
0,2019-09-13,0.2
1,2019-09-13,-0.3
2,2019-09-13,0.0
3,2019-09-13,0.0
4,2019-09-13,0.1
5,2019-09-13,0.3
6,2019-09-13,0.0
7,2019-09-13,0.1
8,2019-09-13,-0.1
9,2019-09-13,0.3
